In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
torch:  2.0 ; cuda:  cu117
detectron2: 0.6


In [2]:
import os
import sys
import logging
import argparse
os.environ["DATASET"] = "../datasets"

pth = '/'.join(sys.path[0].split('/')[:-1])
sys.path.insert(0, pth)

from pprint import pprint
import numpy as np
np.random.seed(1)

home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
print(home_dir)

import warnings
warnings.filterwarnings(action='ignore')
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from hdecoder.BaseModel import BaseModel
from hdecoder import build_model
from utils.distributed import init_distributed
from utils.arguments import load_opt_from_config_files, load_config_dict_to_opt
from utils.misc import MetricLogger

/home/djjin/Mygit/X-Decoder


Invalid MIT-MAGIC-COOKIE-1 key

In [3]:
from utils.arguments import load_vcoco_opt_command, load_vcoco_parser

cmdline_args = load_vcoco_parser()
cmdline_args.conf_files = [os.path.join(home_dir, "configs/hdecoder/vcoco_large.yaml")]

model_path = '../backup/data/output/XDecoder_HOI_230814/Large_90E_0.5904mAP/00234900/default/raw_model_states.pt'
cmdline_args.overrides = ['DONT_LOAD_MODEL', 'false', 'PYLEARN_MODEL', model_path] 

opt = load_vcoco_opt_command(cmdline_args)
opt = init_distributed(opt)

In [4]:
print(opt['base_path'])
print(opt["RESUME"])

../
False


In [5]:
# opt["POSTPROCESS"]["OFFICIAL"]["USE"] = True

In [6]:
from trainer import HDecoder_Trainer as Trainer
if cmdline_args.user_dir:
    absolute_user_dir = os.path.abspath(cmdline_args.user_dir)
    opt['base_path'] = absolute_user_dir
trainer = Trainer(opt)
trainer.eval()

INFO:trainer.distributed_trainer:Setting SAVE_DIR as data/output_large/test
INFO:trainer.distributed_trainer:Using CUDA
INFO:trainer.distributed_trainer:Save config file to data/output_large/test/conf_copy.yaml
INFO:trainer.distributed_trainer:Base learning rate: 0.0001
INFO:trainer.distributed_trainer:Number of GPUs: 1
INFO:trainer.distributed_trainer:Gradient accumulation steps: 1
INFO:trainer.default_trainer:Imported base_dir at base_path ../
INFO:trainer.default_trainer:Pipeline for training: HDecoderPipeline
INFO:trainer.default_trainer:-----------------------------------------------
INFO:trainer.default_trainer:Evaluating model ... 
INFO:base_dir.pipeline.HDecoderPipeline:CDNHOI(
  (backbone): D2FocalNet(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 192, kernel_size=(7, 7), stride=(4, 4), padding=(2, 2))
      (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(

------------------------------------------------------------
               hold_obj: #GTs = 3608, AP = 0.5091
                  stand: #GTs = 4118, AP = 0.7882
              sit_instr: #GTs = 1916, AP = 0.5027
             ride_instr: #GTs = 0556, AP = 0.6184
                   walk: #GTs = 0597, AP = 0.5530
               look_obj: #GTs = 3347, AP = 0.4548
              hit_instr: #GTs = 0349, AP = 0.7687
                hit_obj: #GTs = 0349, AP = 0.4887
                eat_obj: #GTs = 0521, AP = 0.5510
              eat_instr: #GTs = 0521, AP = 0.7217
             jump_instr: #GTs = 0635, AP = 0.7597
              lay_instr: #GTs = 0387, AP = 0.5834
    talk_on_phone_instr: #GTs = 0285, AP = 0.4518
              carry_obj: #GTs = 0472, AP = 0.3973
              throw_obj: #GTs = 0244, AP = 0.4602
              catch_obj: #GTs = 0246, AP = 0.5522
              cut_instr: #GTs = 0269, AP = 0.4372
                cut_obj: #GTs = 0269, AP = 0.6109
                    run: #GTs = 0687, A

{'vcoco_val/vcoco': {'AP_hold_obj': 0.5090548670115597,
  'AP_stand': 0.7882416080874661,
  'AP_sit_instr': 0.5027312937729107,
  'AP_ride_instr': 0.6183787975324447,
  'AP_walk': 0.5530322856132677,
  'AP_look_obj': 0.4547867918568382,
  'AP_hit_instr': 0.7687061978005079,
  'AP_hit_obj': 0.48869914537918924,
  'AP_eat_obj': 0.5510480833788961,
  'AP_eat_instr': 0.7217403534664908,
  'AP_jump_instr': 0.7596972054711459,
  'AP_lay_instr': 0.5834319159478242,
  'AP_talk_on_phone_instr': 0.45179916043552404,
  'AP_carry_obj': 0.39728126084907345,
  'AP_throw_obj': 0.46021073344599184,
  'AP_catch_obj': 0.5521551867971075,
  'AP_cut_instr': 0.43724980886444204,
  'AP_cut_obj': 0.6108790072914593,
  'AP_run': 0.7271945343447281,
  'AP_work_on_computer_instr': 0.6540491794113119,
  'AP_ski_instr': 0.47363009931187783,
  'AP_surf_instr': 0.7459716899645071,
  'AP_skateboard_instr': 0.7689531440786996,
  'AP_smile': 0.7753649028739555,
  'AP_drink_instr': 0.5489129213665838,
  'AP_kick_obj': 